In [ ]:
!pip install autodistill_grounded_sam_2 autodistill

In [1]:
# model_init.py

from autodistill_grounded_sam_2 import GroundedSAM2
from autodistill.detection import CaptionOntology

def initialize_model():
    # Define an ontology to map class names to your Grounded SAM 2 prompt
    ontology = CaptionOntology(
        {
            "person": "person",
            "basketball": "basketball",
            "hoop": "hoop",
            "jersey": "jersey",
            "court lines": "court lines",
        }
    )
    
    # Load the Grounded SAM2 model with the defined ontology
    base_model = GroundedSAM2(ontology=ontology)
    
    return base_model

if __name__ == "__main__":
    model = initialize_model()
    print("Model initialized successfully.")

fatal: destination path 'segment-anything-2' already exists and is not an empty directory.


Obtaining file:///root/.cache/autodistill/segment_anything_2/segment-anything-2
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for SAM-2 (pyproject.toml): started


KeyboardInterrupt: 

In [ ]:
# video_processor.py

import cv2
import yt_dlp
import os
import numpy as np
from autodistill.utils import plot
from model_init import initialize_model

def download_video(url, output_path):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': output_path
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def process_video(video_path, base_model, output_folder):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print(f"Failed to read frame at count: {frame_count}")
            break
        
        # Process every 30th frame (adjust as needed)
        if frame_count % 30 == 0:
            print(f"Processing frame {frame_count}")
            
            # Check if frame is valid
            if frame is None or frame.size == 0:
                print(f"Invalid frame at count: {frame_count}")
                continue
            
            # Save the frame as an image
            frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            
            # Check if frame was saved successfully
            if not os.path.exists(frame_path):
                print(f"Failed to save frame at count: {frame_count}")
                continue
            
            # Run inference on the frame
            try:
                results = base_model.predict(frame_path)
                print(f"Prediction results for frame {frame_count}: {results}")
            except Exception as e:
                print(f"Error during prediction for frame {frame_count}: {str(e)}")
                continue
            
            # Plot the results
            try:
                plot_img = plot(
                    image=frame,
                    classes=base_model.ontology.classes(),
                    detections=results
                )
                print(f"Plot image shape for frame {frame_count}: {plot_img.shape}")
            except Exception as e:
                print(f"Error during plotting for frame {frame_count}: {str(e)}")
                continue
            
            # Check if plot_img is valid
            if plot_img is None or not isinstance(plot_img, np.ndarray) or plot_img.size == 0:
                print(f"Invalid plot image for frame {frame_count}")
                continue
            
            # Save the plotted image
            output_path = os.path.join(output_folder, f"output_{frame_count:04d}.jpg")
            try:
                cv2.imwrite(output_path, plot_img)
                print(f"Successfully saved output for frame {frame_count}")
            except Exception as e:
                print(f"Error saving output for frame {frame_count}: {str(e)}")
        
        frame_count += 1
    
    cap.release()
    print("Video processing complete")

def main():
    video_url = "https://www.youtube.com/watch?v=61jtVJbZAqI"
    output_path = "basketball_video.mp4"
    output_folder = "processed_frames"
    
    # Download the video
    download_video(video_url, output_path)
    
    # Initialize the model
    base_model = initialize_model()
    
    # Process the video
    process_video(output_path, base_model, output_folder)
    print(f"Video processing complete. Processed frames saved in {output_folder}")

if __name__ == "__main__":
    main()